In [ ]:
import pandas as pd

from pandas_datareader import data
import yfinance as yf
from matplotlib import style
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")


import scipy.optimize as sco

yf.pdr_override()

plt.style.use('ggplot')
np.random.seed(42)

portchoice = 8

In [ ]:
#pull in tickers and get rid of ones with weird symbols

df2 = pd.read_csv('C:/Users/danal/Desktop/Tickers101118.csv', header=None)
df2.columns = ['Tickers']
df2 = df2[~df2['Tickers'].str.contains('\^') ]
df2 = df2[~df2['Tickers'].str.contains('\.') ]
df2 = df2[~df2['Tickers'].str.contains('\-') ]
df2 = df2.values.tolist()
len(df2)

In [ ]:
#   My datafetch below   ########################
df3=pd.DataFrame()
df4=pd.DataFrame()

for row in df2:
    try:
        df3=yf.download(row,start="2018-8-16",end="2019-08-16")
        df3adjclose=pd.DataFrame(df3['Adj Close'])
        df3adjclose.columns = [row]
        df4=pd.concat([df4, df3adjclose], axis = 1)
    except:
        pass

In [ ]:
#can use if pickled from spyder
df4 = pd.read_pickle("C:/Users/danal/Desktop/allcomps81919.pkl")
#df4.to_pickle("C:/Users/danal/Desktop/allcomps81919.pkl")

In [ ]:
df4.head(5)

In [ ]:
df4t = df4.T
df4t = df4t[~df4t.index.duplicated(keep='first')]
df4 = df4t.T
df4

In [ ]:
df4rtns_1year = df4.pct_change()
df4rtns_1year.dropna(axis=0, how='all', inplace=True)
df4rtns_1year.dropna(axis=1, how='any', inplace=True)

In [ ]:
df4rtns_1year.head(5)

In [ ]:
#returns and standard deviation for 1 year
df4std_1year = pd.DataFrame(np.std(df4rtns_1year, axis=0), columns = ['Stand_Dev'])
df4std_1year = df4std_1year.reset_index()
df4std_1year.columns = ['Tickers', 'Stand_Dev']
df4std_1year = df4std_1year.set_index('Tickers', drop = True)


df4_avertn_1year = np.mean(df4rtns_1year, axis = 0)
df4_avertn_1year = df4_avertn_1year.reset_index()
df4_avertn_1year.columns = ['Tickers', 'Ave_Return']

In [ ]:
df4_avertn_1year.head(5)

In [ ]:
df4_avertn_1year['Annualized_Rtn'] = df4_avertn_1year["Ave_Return"] * 252

In [ ]:
df4std_1year['Annualized_StdDev'] = df4std_1year['Stand_Dev'] * np.sqrt(252)

In [ ]:
StkDataFrame = df4_avertn_1year.merge(df4std_1year, on="Tickers")
StkDataFrame["Sharpe_Ratio"] = (StkDataFrame["Annualized_Rtn"]-.026)/StkDataFrame["Annualized_StdDev"]
StkDataFrame.head(5)

In [ ]:
StkDataFrame = StkDataFrame[(StkDataFrame["Annualized_StdDev"] >=0) & (StkDataFrame["Annualized_StdDev"] <=.5)]
StkDataFrame = StkDataFrame[(StkDataFrame["Sharpe_Ratio"] >=-3) & (StkDataFrame["Sharpe_Ratio"] <=3)]

In [ ]:
StkDataFrame.describe()

In [ ]:
#sharpe ratio histogram
plt.hist(StkDataFrame['Sharpe_Ratio'], bins=25);

In [ ]:
#Sharpe 1 year where the value is .08 or higher
Sharpegood = StkDataFrame[(StkDataFrame['Sharpe_Ratio']>=1) & (StkDataFrame['Sharpe_Ratio']<=3)]
len(Sharpegood)

In [ ]:
Sharpe1yrlist = Sharpegood['Tickers'].tolist()
Sharpe1yrlist = pd.DataFrame(Sharpe1yrlist)
Sharpe1yrlist.columns = ['Ticker']

In [ ]:
portuniverse=pd.DataFrame()

for x in range(0, 1000):
    Portfolioselect = Sharpe1yrlist.sample(n=portchoice,replace=False)
    Portfolioselect = Portfolioselect.iloc[:, 0].tolist()
    Portfolioselectrtn = df4rtns_1year[Portfolioselect]
    df1yrcovport = Portfolioselectrtn.cov()
    df1yrcovport1 = df1yrcovport.rename_axis(None).rename_axis(None, axis=1)
    df1yrcovport2 = df1yrcovport1.stack().reset_index()
    df1yrcovport2.fillna(value = 0, inplace=True)
    df1yrcovport2['Covariance'] = df1yrcovport2.iloc[:, 2:len(df1yrcovport2)].sum(axis=1)
    df1yrcovport2 = df1yrcovport2[['level_0', 'level_1', 'Covariance']]
    df1yrcovport2 = df1yrcovport2.rename(columns={'level_0': 'Company_A', 'level_1': 'Company_B'})
    #df1yrcovport2 = df1yrcovport2[df1yrcovport2.Covariance != 1]
    

    if (df1yrcovport2['Covariance'] <= .005).all() or (df1yrcovport2['Covariance'] >= -.005).all() :
        Portfolioselect =  pd.DataFrame(Portfolioselect)
        Portfolioselect = Portfolioselect.T
        portuniverse = pd.concat([portuniverse, Portfolioselect], ignore_index = True)
    else:
        pass

#portuniverse.columns = ['stk0','stk1','stk2','stk3','stk4','stk5','stk6','stk7']    
portuniverse = portuniverse.reset_index(drop = True)

In [ ]:
portuniverse.head(5)

In [ ]:
#look for duplicate portfolios

"""portuniverse['stklist'] = portuniverse[['stk0','stk1','stk2','stk3','stk4','stk5','stk6','stk7']].values.tolist()
portuniverse['stklist'] = portuniverse['stklist'].sort_values().apply(lambda x: sorted(x))
portuniverse['totlist'] = ['_'.join(map(str, i)) for i in portuniverse['stklist']]
portuniverse = portuniverse.drop_duplicates(subset='totlist', keep='first')
portuniverse = portuniverse.drop(['stklist', 'totlist'], axis = 1)
portuniverse = portuniverse.reset_index(drop = True)
portuniverse"""

In [ ]:
import numpy as np, numpy.random

Portfoliolist = pd.DataFrame()
Weightingfin = pd.DataFrame()
Portfoliodetail = pd.DataFrame(columns = ["Tickers", "Weights", "Returns", "Stand_Dev", "Portfolio_Num", "Iteration"])

for row in range(0,len(portuniverse)):
    portfoliox = portuniverse.loc[row]
    portfolioxdfi = pd.DataFrame(portfoliox)
    portfolioxdfi.columns = ['Tickers']
    
    for x in range (0,100):
        weightsdf = pd.DataFrame(np.random.dirichlet(np.ones(portchoice),size=1))
        weightsdf = round(weightsdf,3)
        weightsdf = weightsdf.T
        weightsdf.columns = ['Weights']
        portfolioxdfi2 = pd.concat([portfolioxdfi, weightsdf], axis = 1)
        portfolioxdfirtn = pd.merge(portfolioxdfi2, df4_avertn_1year, on='Tickers', how='left')
        portfolioxdfirtn = pd.merge(portfolioxdfirtn, df4std_1year, on='Tickers', how='left')
        portfolioxdfirtn['Portfolio_Num'] = row
        portfolioxdfirtn['Iteration'] = x
        
        Portfoliodetail = pd.concat([Portfoliodetail, portfolioxdfirtn], axis = 0)
        
        portfolioxdfirtnval = portfolioxdfirtn['Weights'] * portfolioxdfirtn['Annualized_Rtn']
        portfolioxdfirtnvalf = np.sum(portfolioxdfirtnval)
        
        Portfolioselectsp = portfolioxdfirtn.iloc[:, 0].tolist()
        Portfolioselectrtns = df4rtns_1year[Portfolioselectsp]
        Portfolioselectcovm = Portfolioselectrtns.cov()
        Portfoliocovariance = np.dot(portfolioxdfirtn['Weights'].T, Portfolioselectcovm)
        Portfoliocovariance = np.sqrt(np.dot(Portfoliocovariance, portfolioxdfirtn['Weights']))
        SharpeRatio = ((portfolioxdfirtnvalf - .026)/(np.sqrt(Portfoliocovariance)))
        portmetrics = pd.DataFrame([row, portfolioxdfirtnvalf, Portfoliocovariance, SharpeRatio]).T
        Portfoliolist = pd.concat([Portfoliolist,portmetrics], axis = 0)
        Weighting = weightsdf.T
        Weighting['Portfolio'] = row
        Weighting['Iteration'] = x
        Weighting['Portfolio_Covariance'] = Portfoliocovariance
        Weightingfin = pd.concat([Weightingfin, Weighting], axis = 0)

Portfoliolist.columns = ['Portfolio_Number', 'Portfolio_Return', 'Portfolio_Covariance', 'Sharpe_Ratio']
Weightingfin = Weightingfin.reset_index(drop = True)

In [ ]:
df4 = pd.read_pickle("C:/Users/danal/Desktop/portfoliolist81919.pkl")
#df4.to_pickle("C:/Users/danal/Desktop/portfoliolist81919.pkl")

#!pip install palettable

plt.subplots(figsize=(20,15))

sns.scatterplot(x="Portfolio_Covariance", y="Portfolio_Return",
                hue="Sharpe_Ratio",
                palette='plasma_r',
                linewidth=0,
                data=Portfoliolist);

In [ ]:
Portfoliolist.shape

In [ ]:
MinimumVar = Portfoliolist[Portfoliolist['Portfolio_Covariance'] == min(Portfoliolist['Portfolio_Covariance'])]
MinimumVar

In [ ]:
MinimumVarPort = MinimumVar['Portfolio_Number'].astype(int)
MinimumVarPort2 = portuniverse.iloc[MinimumVarPort]
MinimumVarPort2

In [ ]:
#Portfoliolist.to_pickle('D:/NewMexicoData/Portlist.pkl')
#Weightingfin.to_pickle('D:/NewMexicoData/Weightingfinlist.pkl')

In [ ]:
Weightingfin[Weightingfin['Portfolio_Covariance'] == min(Weightingfin['Portfolio_Covariance'])]